<a href="https://colab.research.google.com/github/joemarshall/websensors/blob/main/assets/python/BinaryClassifier" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab workbook presents a workflow for training a machine learning model for a simple classifier, then outputs it as a tflite file which can be used in the websensor platform or on a raspberry pi.

In [ ]:
# tensorflow is the machine learning library we use
import tensorflow as tf
# numpy is for fast python maths
import numpy as np
# pandas for importing datafiles
import pandas as pd
import io

# make some stuff that is in tensorflow be 
# easier to get at below
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.losses as losses




In [ ]:
# load datafiles - each datafile is a csv file of continuous sensor data with. 
# accompanying ground truth tempo data calculated from the previous 4 beats

# recorded data can be from a raspberry pi or from the websensor platform

# this stuff makes an upload box appear
from google.colab import files
uploaded = files.upload()

Saving tempo_gt.csv to tempo_gt (1).csv


In [ ]:
# preprocess - for each data point, add history of the previous 511 points
# this is called 'unrolling'
def unroll_data_and_preprocess(data,gt):
  np_data=np.array(data)
  np_data=(np_data/512.0) # scale the data so it isn't too big
  return (np.lib.stride_tricks.sliding_window_view(np_data,window_shape=[512])).copy(),gt[511:]



column_names=["sound level","ground truth"]

datasets=[]

for c in uploaded.keys():
  print(f"Loading: {c}")
  csv_frame=pd.read_csv(io.BytesIO(uploaded[c]))
  file_x = csv_frame[column_names[0]].to_numpy()
  file_y= csv_frame[column_names[1]].to_numpy()
  datasets.append(unroll_data_and_preprocess(file_x,file_y))

# make arrays for x and y
x_data=np.concatenate([x for (x,y) in datasets])
y_data=np.concatenate([y for (x,y) in datasets])
print(f"Loaded data: {x_data.shape},{y_data.shape}")

Loading: tempo_gt.csv
Loaded data: (7950, 512),(7950,)


In [ ]:
# shuffle the datasets
p = np.random.permutation(x_data.shape[0])
x_data=x_data[p]
y_data=y_data[p]

# split the datasets into train and test
split_point=int (x_data.shape[0]*.75 )
#split_point=x_data.shape[0]-1
x_train=x_data[0:split_point]
x_test=x_data[split_point:]
y_train=y_data[0:split_point]
y_test=y_data[split_point:]




In [ ]:
# build a model - 4 convolutional layers to identify features, then a fully connected layer to output 
# the tempo value
model=keras.Sequential(layers=[layers.Input(name='x',shape=(512,1)),layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         layers.Conv1D(32,kernel_size=3,padding="same",strides=2,activation="relu"),
                         #layers.Conv1D(64,kernel_size=32,padding="same",strides=32,activation="relu"),
                         layers.Flatten(),
                         layers.Dense(64,activation="relu"),
                         layers.Dense(1,activation="sigmoid",name='y')]) # sigmoid = between 0 and 1
model.compile(optimizer='adam', loss='mse',run_eagerly=True) # mean squared error loss is a good default for regression problems
model.build(input_shape=(None,512,1))
model.summary()
print(model.input,model.output)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, 256, 32)           128       
                                                                 
 conv1d_9 (Conv1D)           (None, 128, 32)           3104      
                                                                 
 conv1d_10 (Conv1D)          (None, 64, 32)            3104      
                                                                 
 conv1d_11 (Conv1D)          (None, 32, 32)            3104      
                                                                 
 flatten_2 (Flatten)         (None, 1024)              0         
                                                                 
 dense_2 (Dense)             (None, 64)                65600     
                                                                 
 y (Dense)                   (None, 1)                

In [ ]:
# call train on the model
model.fit(x_train,y_train,batch_size=32,validation_data=(x_test,y_test),epochs=100)


Epoch 1/100
187/187 [==============================] - 5s 29ms/step - loss: 707.5798 - val_loss: 642.1845
Epoch 2/100
187/187 [==============================] - 6s 30ms/step - loss: 605.3569 - val_loss: 546.2111
Epoch 3/100
187/187 [==============================] - 6s 30ms/step - loss: 520.2805 - val_loss: 491.4795
Epoch 4/100
187/187 [==============================] - 5s 29ms/step - loss: 464.1988 - val_loss: 490.2721
Epoch 5/100
187/187 [==============================] - 5s 29ms/step - loss: 375.9774 - val_loss: 361.6880
Epoch 6/100
187/187 [==============================] - 6s 30ms/step - loss: 352.3999 - val_loss: 459.3267
Epoch 7/100
187/187 [==============================] - 6s 30ms/step - loss: 336.7682 - val_loss: 315.1294
Epoch 8/100
187/187 [==============================] - 6s 33ms/step - loss: 293.3651 - val_loss: 268.4543
Epoch 9/100
187/187 [==============================] - 5s 28ms/step - loss: 265.6620 - val_loss: 265.4473
Epoch 10/100
187/187 [========================

In [ ]:
# Save model to a tflite model for inference on raspberry pi (or websensor platform)
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite=converter.convert()

tflite_model_file = open('model.tflite',"wb")
tflite_model_file.write(tflite)

interpreter = tf.lite.Interpreter(model_content=tflite)

signatures = interpreter.get_signature_list()
print(signatures)

from google.colab import files
files.download('model.tflite')


INFO:tensorflow:Assets written to: /tmp/tmp1w1_nmr7/assets


{'serving_default': {'inputs': ['x'], 'outputs': ['y']}}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>